In [ ]:
# ============================================================
# 🏭 Capacidade Produtiva — Layout Final (cards + gráficos + tabelas)
# ============================================================

from dataclasses import dataclass
from typing import Optional, Tuple
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import fill
from IPython.display import display, HTML

# -----------------------------
# 1) Entrada e validação
# -----------------------------
def _to_float(x: str) -> float:
    x = x.strip().replace(",", ".")
    return float(x)

def perguntar_float(msg: str, minimo: Optional[float] = None) -> float:
    while True:
        try:
            v = _to_float(input(msg))
            if minimo is not None and v < minimo:
                print(f"Valor deve ser >= {minimo}. Tente novamente.")
                continue
            return v
        except ValueError:
            print("Entrada inválida (use número, ex.: 2.5 ou 2,5). Tente novamente.")

def perguntar_int(msg: str, minimo: Optional[int] = None) -> int:
    while True:
        try:
            v = int(input(msg))
            if minimo is not None and v < minimo:
                print(f"Valor deve ser >= {minimo}. Tente novamente.")
                continue
            return v
        except ValueError:
            print("Entrada inválida (use número inteiro). Tente novamente.")

def coletar_parametros_interativos() -> Tuple[float, int, float]:
    print("=== Parâmetros Operacionais ===")
    vel = perguntar_float("Velocidade do equipamento (kg/h): ", minimo=0.0001)
    turnos = perguntar_int("Quantidade de turnos: ", minimo=1)
    horas_turno = perguntar_float("Horas por turno: ", minimo=0.25)
    return vel, turnos, horas_turno

def registrar_ocorrencias(tipo: str) -> pd.DataFrame:
    print(f"\n--- Registrar ocorrências {tipo}(s) ---")
    itens = []
    while True:
        nome = input("Descrição da ocorrência (ou 'fim' p/ encerrar): ").strip()
        if nome.lower() == "fim":
            break
        horas = perguntar_float(f"Tempo perdido (h) em '{nome}': ", minimo=0)
        itens.append({"Ocorrencia": nome, "Horas": horas})
    df = pd.DataFrame(itens) if itens else pd.DataFrame(columns=["Ocorrencia", "Horas"])
    return df

# -----------------------------
# 2) Cálculo de capacidades
# -----------------------------
@dataclass
class ResultadoCapacidade:
    cap_instalada_h: float
    cap_instalada_kg: float
    cap_disponivel_h: float
    cap_disponivel_kg: float
    cap_efetiva_h: float
    cap_efetiva_kg: float
    cap_realizada_h: float
    cap_realizada_kg: float
    grau_dispon: float
    grau_util: float
    indice_ef: float
    perdas_plan_h: float
    perdas_nplan_h: float
    df_plan: pd.DataFrame
    df_nplan: pd.DataFrame

def calcular_capacidades(
    velocidade_kg_h: float,
    turnos: int,
    horas_por_turno: float,
    df_plan: pd.DataFrame,
    df_nplan: pd.DataFrame,
    dias_instalada: int = 7,
    dias_disponivel: int = 5
) -> ResultadoCapacidade:

    cap_instalada_h = dias_instalada * 24
    cap_disponivel_h = turnos * horas_por_turno * dias_disponivel

    perdas_plan_h = float(df_plan["Horas"].sum()) if not df_plan.empty else 0.0
    perdas_nplan_h = float(df_nplan["Horas"].sum()) if not df_nplan.empty else 0.0

    cap_efetiva_h = max(cap_disponivel_h - perdas_plan_h, 0.0)
    cap_realizada_h = max(cap_efetiva_h - perdas_nplan_h, 0.0)

    cap_instalada_kg = cap_instalada_h * velocidade_kg_h
    cap_disponivel_kg = cap_disponivel_h * velocidade_kg_h
    cap_efetiva_kg = cap_efetiva_h * velocidade_kg_h
    cap_realizada_kg = cap_realizada_h * velocidade_kg_h

    grau_dispon = (cap_disponivel_kg / cap_instalada_kg * 100) if cap_instalada_kg > 0 else 0.0
    grau_util   = (cap_efetiva_kg   / cap_disponivel_kg * 100) if cap_disponivel_kg > 0 else 0.0
    indice_ef   = (cap_realizada_kg / cap_efetiva_kg   * 100) if cap_efetiva_kg   > 0 else 0.0

    return ResultadoCapacidade(
        cap_instalada_h, cap_instalada_kg,
        cap_disponivel_h, cap_disponivel_kg,
        cap_efetiva_h, cap_efetiva_kg,
        cap_realizada_h, cap_realizada_kg,
        grau_dispon, grau_util, indice_ef,
        perdas_plan_h, perdas_nplan_h, df_plan, df_nplan
    )

# -----------------------------
# 3) Pré-processo das ocorrências
# -----------------------------
def preparar_ocorrencias(df: pd.DataFrame, titulo: str) -> pd.DataFrame:
    df2 = df.copy()
    if df2.empty:
        return pd.DataFrame(columns=["Ocorrencia", "Horas", "Percentual (%)"])
    total = df2["Horas"].sum()
    df2["Percentual (%)"] = np.where(total > 0, (df2["Horas"] / total) * 100, 0.0)
    df2 = df2.sort_values("Horas", ascending=False).reset_index(drop=True)
    total_row = pd.DataFrame({"Ocorrencia": [f"TOTAL {titulo}"], "Horas": [total], "Percentual (%)": [100.0 if total > 0 else 0.0]})
    df2 = pd.concat([df2, total_row], ignore_index=True)
    return df2

# -----------------------------
# 4) DataFrame de resultados
# -----------------------------
def montar_dataframe_resultados(r: ResultadoCapacidade) -> pd.DataFrame:
    return pd.DataFrame({
        "Indicador": [
            "Capacidade Instalada (kg)",
            "Capacidade Disponível (kg)",
            "Capacidade Efetiva (kg)",
            "Capacidade Realizada (kg)",
            "Grau de Disponibilidade (%)",
            "Grau de Utilização (%)",
            "Índice de Eficiência (%)",
        ],
        "Valor": [
            r.cap_instalada_kg, r.cap_disponivel_kg,
            r.cap_efetiva_kg, r.cap_realizada_kg,
            r.grau_dispon, r.grau_util, r.indice_ef
        ],
        "Tipo": ["Capacidade","Capacidade","Capacidade","Capacidade","Índice","Índice","Índice"]
    })

# -----------------------------
# 5) UI — Cards e Gráficos
# -----------------------------
def _kpi_card(ax, title, value, subtitle=None):
    ax.axis("off")
    ax.set_xlim(0, 1); ax.set_ylim(0, 1)
    ax.text(0.05, 0.72, title, fontsize=10, fontweight="bold")
    ax.text(0.05, 0.40, value, fontsize=16, fontweight="bold")
    if subtitle:
        ax.text(0.05, 0.18, subtitle, fontsize=9, color="dimgray")
    rect = plt.Rectangle((0.02,0.06), 0.96, 0.88, fill=False, lw=1, ec="lightgray")
    ax.add_patch(rect)

def exibir_kpis(r: ResultadoCapacidade):
    fig, axs = plt.subplots(1, 3, figsize=(12, 2.8))
    _kpi_card(axs[0], "Disponibilidade", f"{r.grau_dispon:,.2f} %", "Disponível / Instalada")
    _kpi_card(axs[1], "Utilização",      f"{r.grau_util:,.2f} %",   "Efetiva / Disponível")
    _kpi_card(axs[2], "Eficiência",      f"{r.indice_ef:,.2f} %",   "Realizada / Efetiva")
    fig.suptitle("Indicadores‑Chave de Desempenho (KPI)", y=1.08, fontsize=12, fontweight="bold")
    plt.tight_layout()
    plt.show()

def graficos_principais_lado_a_lado(df_res: pd.DataFrame):
    def wrap_labels(labels, width=16):
        return [fill(lbl, width=width) for lbl in labels]
    fig, axes = plt.subplots(1, 2, figsize=(12, 4.2))
    fig.suptitle("Visão Geral — Capacidades e Índices", fontsize=12, fontweight="bold", y=1.05)

    # Capacidades
    caps = df_res[df_res["Tipo"]=="Capacidade"].copy()
    axes[0].bar(range(len(caps)), caps["Valor"])
    axes[0].set_title("Capacidades (kg)", pad=12)
    axes[0].set_ylabel("Valor (kg)")
    axes[0].set_xticks(range(len(caps)))
    axes[0].set_xticklabels(wrap_labels(caps["Indicador"].tolist(), width=18), rotation=15, ha="right")
    for i, v in enumerate(caps["Valor"]):
        axes[0].text(i, v, f"{v:,.0f}", ha="center", va="bottom", fontsize=9)

    # Índices
    inds = df_res[df_res["Tipo"]=="Índice"].copy()
    axes[1].bar(range(len(inds)), inds["Valor"])
    axes[1].set_title("Índices (%)", pad=12)
    axes[1].set_ylabel("Percentual")
    axes[1].set_ylim(0, 100)
    axes[1].set_xticks(range(len(inds)))
    axes[1].set_xticklabels(wrap_labels(inds["Indicador"].tolist(), width=18), rotation=15, ha="right")
    for i, v in enumerate(inds["Valor"]):
        axes[1].text(i, v+1, f"{v:,.2f}%", ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.show()

def graficos_ocorrencias_lado_a_lado(df_plan: pd.DataFrame, df_nplan: pd.DataFrame):
    def prepare_plot_df(df):
        if df.empty:
            return df
        if len(df) > 1 and str(df.iloc[-1, 0]).startswith("TOTAL"):
            return df.iloc[:-1]
        return df

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle("Ocorrências (horas)", fontsize=12, fontweight="bold", y=1.03)

    # Planejadas
    d1 = prepare_plot_df(df_plan)
    if d1.empty:
        axes[0].text(0.5, 0.5, "Sem registros", ha="center", va="center")
        axes[0].axis("off")
    else:
        axes[0].bar(range(len(d1)), d1["Horas"])
        axes[0].set_title("Planejadas", pad=10)
        axes[0].set_ylabel("Horas")
        axes[0].set_xticks(range(len(d1)))
        axes[0].set_xticklabels([fill(x, 18) for x in d1["Ocorrencia"]], rotation=15, ha="right")
        for i, v in enumerate(d1["Horas"]):
            axes[0].text(i, v, f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

    # Não Planejadas
    d2 = prepare_plot_df(df_nplan)
    if d2.empty:
        axes[1].text(0.5, 0.5, "Sem registros", ha="center", va="center")
        axes[1].axis("off")
    else:
        axes[1].bar(range(len(d2)), d2["Horas"])
        axes[1].set_title("Não Planejadas", pad=10)
        axes[1].set_ylabel("Horas")
        axes[1].set_xticks(range(len(d2)))
        axes[1].set_xticklabels([fill(x, 18) for x in d2["Ocorrencia"]], rotation=15, ha="right")
        for i, v in enumerate(d2["Horas"]):
            axes[1].text(i, v, f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

    plt.tight_layout()
    plt.show()

# -----------------------------
# 6) Tabelas finais estilizadas e em coluna
# -----------------------------
def mostrar_tabela_com_titulo(df: pd.DataFrame, titulo: str, formato: dict):
    styled = df.style.format(formato).set_caption(titulo)
    html = styled.to_html()
    return f"<div style='margin-bottom:32px'>{html}</div>"

def mostrar_tabelas_finais_em_coluna(df_plan, df_nplan, df_res):
    html1 = mostrar_tabela_com_titulo(df_plan, "Ocorrências Planejadas (horas e %)", {"Horas": "{:,.2f}", "Percentual (%)": "{:,.2f}"})
    html2 = mostrar_tabela_com_titulo(df_nplan, "Ocorrências Não Planejadas (horas e %)", {"Horas": "{:,.2f}", "Percentual (%)": "{:,.2f}"})
    html3 = mostrar_tabela_com_titulo(df_res[["Indicador","Valor"]], "Indicador x Valor (resultado final)", {"Valor": "{:,.2f}"})

    html_final = f"""
    <div style='display:flex; flex-direction:column; gap:36px;'>
      {html1}
      {html2}
      {html3}
    </div>
    """
    display(HTML(html_final))

# -----------------------------
# 7) Execução (interativa)
# -----------------------------
if __name__ == "__main__":
    # Entradas
    velocidade, turnos, horas_turno = coletar_parametros_interativos()
    # Ocorrências
    df_plan_raw  = registrar_ocorrencias("planejada")
    df_nplan_raw = registrar_ocorrencias("não planejada")
    # Cálculo
    r = calcular_capacidades(velocidade, turnos, horas_turno, df_plan_raw, df_nplan_raw)
    # Preparar ocorrências com total e percentual
    df_plan  = preparar_ocorrencias(r.df_plan,  "PLANEJADAS")
    df_nplan = preparar_ocorrencias(r.df_nplan, "NÃO PLANEJADAS")
    # Resultados agregados
    df_res = montar_dataframe_resultados(r)

    # 1) Cards KPI
    exibir_kpis(r)
    # 2) Gráficos principais lado a lado — capacidades e índices
    graficos_principais_lado_a_lado(df_res)
    # 3) Gráficos de ocorrências lado a lado
    graficos_ocorrencias_lado_a_lado(df_plan, df_nplan)
    # 4) Tabelas finais - uma abaixo da outra, com títulos e espaçamento
    mostrar_tabelas_finais_em_coluna(df_plan, df_nplan, df_res)
